### Multiple Workers

In the last example in the testing folder we had one worker only to consume all the tasks. To scale the system we will need multiple workers to share the tasks. We will be creating multiple workers and they will be consuming the in a round robin fashion. That helps in parallelizing the tasks to all the worker nodes  done internally by RabbitMQ

To mock the task we will be using time.sleep based on number of dots each message has. So a task message "message..." will take 3 seconds to finish and so on. Let's make the changes to the code

In [64]:
#send.py
# Producer takes in an argument
import sys
import pika

connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()
channel.queue_declare(queue='hello')
def sendQueue():
    try:
        while True:
            # keep taking input till kernel is interrupted
            message = input("Enter new task")
            channel.basic_publish(exchange='',
                                  routing_key='hello',
                                  body=message)
            print(" [x] Sent %r" % message)
    except KeyboardInterrupt:
        # Exit gracefully
        connection.close()

In [65]:
sendQueue()

Enter new taskm.
 [x] Sent 'm.'
Enter new taskm..
 [x] Sent 'm..'
Enter new taskm...
 [x] Sent 'm...'
Enter new taskm....
 [x] Sent 'm....'
Enter new taskm.....
 [x] Sent 'm.....'
Enter new taskm......
 [x] Sent 'm......'


In [46]:
# recieve.py
# consumer takes in the message and counts the dots in it to mock the task duration. It sleeps for seconds as much dots are in the message
import time
import pika

connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
channel = connection.channel()
channel.queue_declare(queue='hello')

def callback(ch, method, properties, body):
    print(" [x] Received %r" % body.decode())
    time.sleep(body.count(b'.'))
    print(" [x] Done")
    
channel.basic_consume(queue='hello', auto_ack=True, on_message_callback=callback)

try:
    channel.start_consuming()
except KeyboardInterrupt:
    # Exit gracefully
    connection.close()

'ctag1.ec88106dcaad42f5b03e1fd2f8383dd3'

We will need 3 shells to run this thing. Two worker nodes and one producer node. We will use this notebook as the producer

So the tasks got divided among both the workers equally. The good thing about round robin approach is its parallize the task load. If our task queue gets overwhelming just add new worker node and the task gets split among them

Now let's look at the downside of this approach. The round robin distributes tasks in a sequence and marks for deletion. So what happens when one of the worker node working on a time consuming task goes down?

The task doesn't get completed and the pending task on that worker node which were dispatched and marked for deletion by the producer node also gets removed before completion.

#### Message ACKnowledgement to the rescue

The concept is simple. The rabbitmq server will wait for an acknowledgement from each of the worker nodes for each task that the task is consumed and processed so rabbitmq is free to delete it.

In [ ]:
Suppose one of the worker nodes goes down for all the reason a node can go down, rabbitmq server will still be listening for the acknowledgement and after waiting for a set period of time (timeout) rabbitmq re-queue the task and distribute it to 